In [1]:
import numpy as np
import pandas as pd

import seaborn as sns  
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["font.sans-serif"]=["SimHei"]#展示中文字体
mpl.rcParams["axes.unicode_minus"]=False#处理负刻度值

In [2]:
product_type_raw = pd.read_excel("data/附件2-商品信息表.xlsx")
storage_type_raw = pd.read_excel("data/附件4-仓库信息表.xlsx")
seller_type_raw = pd.read_excel("data/附件3-商家信息表.xlsx")
seller_type_raw.head()

,seller_no,seller_category,inventory_category,seller_level
0,seller_1,居家生活,B,New
1,seller_2,厨具,A,Large
2,seller_3,厨具,A,Large
3,seller_4,家居日用,A,Large
4,seller_5,家居日用,B,Large


In [3]:
new_data_raw = pd.read_excel("data/附件5-新品历史出货量表.xlsx")

In [110]:
data = new_data_raw.copy()
data["qty"] = data["qty"].where(data["qty"] <= 200, 200)
data.head()

,seller_no,warehouse_no,product_no,date,qty
0,seller_19,wh_21,product_2215,2023-05-08,2
1,seller_19,wh_21,product_2215,2023-04-11,2
2,seller_19,wh_21,product_2215,2023-05-14,22
3,seller_19,wh_21,product_2215,2023-04-15,1
4,seller_19,wh_21,product_2215,2023-04-29,3


In [111]:
product_type = product_type_raw.set_index("product_no")
storage_type = storage_type_raw.drop_duplicates(keep='first').set_index("warehouse_no")
seller_type = seller_type_raw.set_index("seller_no")
seller_type["mixed type"] = seller_type["inventory_category"]+" "+seller_type["seller_level"]
seller_type.head()

,seller_category,inventory_category,seller_level,mixed type
seller_no,,,,
seller_1,居家生活,B,New,B New
seller_2,厨具,A,Large,A Large
seller_3,厨具,A,Large,A Large
seller_4,家居日用,A,Large,A Large
seller_5,家居日用,B,Large,B Large


In [112]:
data["seller type"] = seller_type.loc[data["seller_no"]]["mixed type"].values

# 模型读取+分类

In [45]:
# 数据集划分，自定义窗口大小和数据量

product_num = 1000
ratio = 0.8

WINDOW = 10
FEATURES = 9

from numpy.lib.stride_tricks import sliding_window_view
def split_data(raw, window, num=product_num):
    
    index = raw.index.get_level_values(0).unique()
    print(len(index))
    num = min(num, len(index))
    
    def choice(index, count):
        index = np.random.choice(index, count, replace=False)
        return index
    
    prod_choice = choice(index, num)
    data = raw.loc[prod_choice]
    print(data.head())
    data = data.to_numpy()
    print(data.shape)
    
    x = sliding_window_view(data, window_shape=(window,FEATURES)).squeeze(1)[:-1].reshape((-1, window*FEATURES)) 
    y = data.T[0, window:]
    
    train_count = int(ratio * len(y))
    
    return x[:train_count], y[:train_count], x[train_count:], y[train_count:]

In [155]:
# 滑动窗口预测

def LongTerm(model, step, data_x, first_day=0):
    
    shape_flat = np.prod(data_x.shape)
    
    if shape_flat%(WINDOW*FEATURES) != 0:
        return None
    if shape_flat == 0:
        return None
    
    root = data_x.reshape(-1, WINDOW, FEATURES).squeeze(0)
    
    for i in range(step):
        predict_window = root[-WINDOW:]
        series = root.T[0][-WINDOW:]
        
        predict = model.predict(predict_window.reshape(1, WINDOW*FEATURES))[0]
        predict = max(min(predict, 200), 0)
        series = np.append(series, predict)
        
        mean_5 = series[-5:].mean()
        std_5 = series[-5:].std()
        mean_15 = series[-15:].mean()
        std_15 = series[-15:].std()
        min_15 = series[-15:].min()
        max_15 = series[-15:].max()
        
        d_mean_5 = mean_5 - predict_window.T[1][-1]
        weekend = first_day%7 in [5,6] 
        
        new_col = [predict, mean_5, std_5, mean_15, std_15, min_15, max_15, d_mean_5, weekend]
        #print(new_col)
        
        root = np.append(root, [new_col], axis = 0)
        
    return root.T

In [126]:
# 1-wmape 打分

def WmapeScore(predict, gt):
    gt_sum = gt.sum()
    err_sum = np.abs(predict-gt).sum()
    
    return 1 - err_sum/gt_sum

In [147]:
# 训练单元

class TrainingUnit:
    
    def __init__(self, data, window, num, model):
        
        if data is not None:
            self.train_x, self.train_y, self.test_x, self.test_y = split_data(data, window, num)
        self.model = model
        
        self.window = window
        
    def train(self):
        
        self.model.fit(self.train_x, self.train_y)
        
    def evalueate_single(self, type_name=None, show_graph=True):
        
        predict = self.model.predict(self.test_x)
        
        if show_graph:
            plt.plot(self.test_y[:300], label='真实值')
            plt.plot(predict[:300],     label='预测值')
            plt.legend()
            plt.xlabel('日期')
            plt.ylabel('需求量' if type_name is None else type_name+"型商家需求量")
            
            plt.subplots_adjust(left=0, right=2)
            plt.show()
        
        mse = np.mean((self.test_y - predict) ** 2)
        print("MSE:",mse)
        print("MIN:",np.mean(np.diff(self.test_y)**2))
        
        return predict, mse
        
    def long_term_predict(self, step, root=None, first_day=0):
        if root is None:
            root = self.test_x[self.window:][0]
            
        result = LongTerm(self.model, step, root, first_day)
        
        return result, root
    
    def evalueate(self, step, predict, real): 
        if predict is None:
            return 0
        
        w_score = WmapeScore(predict[0][self.window:], real[self.window:self.window+step])
        
        return w_score
    
    def evalueate_long(self, step, times=1):
        w_scores = []
        for i in range(1):
            high = len(self.test_x)-self.window*3
            
            offset = np.random.randint(0, min(max(1, high),len(self.test_x)))
            
            if self.test_x[offset+self.window:].shape[0] == 0:
                continue
                
            root = self.test_x[offset+self.window:][0]
            predict, _ = self.long_term_predict(step, root)
            
            if predict is None:
                continue
            
            w_score = self.evalueate(step, predict, self.test_y[offset:])
            w_scores += [w_score]
            
        return np.mean(w_scores)
        
    def show_long(self, result, root, step=15, type_name=None):
        
        plt.plot(root[:self.window+step], label="GT")
        plt.plot(result[0], label="Predict")
        plt.legend()
        
        plt.subplots_adjust(left=0, right=2)
        plt.show()

In [148]:
import joblib
import os
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

LinearModels = RFModels = dict()

for root, dirs, files in os.walk("models"):
    print(root, dirs)
    for file in files:
        model_type = file[:-7]
        model = joblib.load("models/"+file)
        
        df = data.groupby("seller type").get_group(model_type).sort_values(by=["product_no","warehouse_no", "date"])
        df = df[["product_no","date","qty"]].set_index(["product_no","date"])
        
        df["mean 5"] = df["qty"].rolling(window=5).mean().fillna(0)
        df["std 5"] = df["qty"].rolling(window=5).std().fillna(0)
        df["mean 15"] = df["qty"].rolling(window=15).mean().fillna(0)
        df["std 15"] = df["qty"].rolling(window=15).std().fillna(0)
        df["min 15"] = df["qty"].rolling(window=15).min().fillna(0)
        df["max 15"] = df["qty"].rolling(window=15).max().fillna(0)
        df["d mean 5"] = df["mean 5"].diff().fillna(0)
        df["weekend"] = pd.to_datetime(df.index.get_level_values(1)).dayofweek.isin([5, 6])
        
        print(df.shape)
        
        if "_LR" in file:
            LinearModels[model_type] = TrainingUnit(df, 10, 1000, model)
        elif "_RF" in file:
            RFModels[model_type] = TrainingUnit(df, 10, 10, model)

models []
(1319, 9)
30
                         qty  mean 5     std 5   mean 15    std 15  min 15  \
product_no   date                                                            
product_2133 2023-04-06    2     1.8  0.836660  0.733333  1.032796     0.0   
             2023-04-07    0     1.6  1.140175  0.733333  1.032796     0.0   
             2023-04-08    0     1.2  1.303840  0.733333  1.032796     0.0   
             2023-04-09    0     1.0  1.414214  0.733333  1.032796     0.0   
             2023-04-10    0     0.4  0.894427  0.733333  1.032796     0.0   

                         max 15  d mean 5  weekend  
product_no   date                                   
product_2133 2023-04-06     3.0       0.4    False  
             2023-04-07     3.0      -0.2    False  
             2023-04-08     3.0      -0.4     True  
             2023-04-09     3.0      -0.2     True  
             2023-04-10     3.0      -0.6    False  
(1319, 9)
(1319, 9)
30
                         qty  mean 5

(740, 9)
13
                         qty  mean 5     std 5   mean 15    std 15  min 15  \
product_no   date                                                            
product_2122 2023-04-07    1     1.8  1.303840  2.133333  1.726543     0.0   
             2023-04-08    1     1.8  1.303840  2.133333  1.726543     0.0   
             2023-04-09    1     1.8  1.303840  2.200000  1.656157     1.0   
             2023-04-10    0     1.0  0.707107  2.000000  1.732051     0.0   
             2023-04-11    1     0.8  0.447214  1.733333  1.533747     0.0   

                         max 15  d mean 5  weekend  
product_no   date                                   
product_2122 2023-04-07     6.0      -1.0    False  
             2023-04-08     6.0       0.0     True  
             2023-04-09     6.0       0.0     True  
             2023-04-10     6.0      -0.8    False  
             2023-04-11     6.0      -0.2    False  
(541, 9)
(505, 9)
10
                         qty  mean 5     std 5   

KeyError: 'D Medium'

## 自主预测

### 微调

In [149]:
def TrainingProcess(table):

    count = 0
    for t, unit in table.items():
        unit.train()
        count+=1
        print(f"微调：{count}/{len(table)}")
        
    for t, unit in table.items():
        score = unit.evalueate_long(15, 30)
        print(t, score)


In [150]:
TrainingProcess(LinearModels)
TrainingProcess(RFModels)

微调：1/9
微调：2/9
微调：3/9
微调：4/9
微调：5/9
微调：6/9
微调：7/9
微调：8/9
微调：9/9
(90,)
A Large -2.6979999999999995
A Medium nan
(90,)
A Special 0.5426819923371647
(90,)
B Large -3.673333333333334
(90,)
B Medium 0.1661538461538462
(90,)
B Small 0.18780487804878054
(90,)
B Special -4.303333333333334
(90,)
C Large -0.1882222222222223
(90,)


D:\ProgramData\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\ProgramData\anaconda3\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


D Large -1.6104444444444446
微调：1/9
微调：2/9
微调：3/9
微调：4/9
微调：5/9
微调：6/9
微调：7/9
微调：8/9
微调：9/9
(90,)
A Large -4.6571428571428575
A Medium nan
(90,)
A Special 0.5426819923371647
(90,)
B Large -1.1124999999999998
(90,)
B Medium 0.33904761904761904
(90,)
B Small 0.38296296296296295
(90,)


D:\ProgramData\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\ProgramData\anaconda3\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


B Special -4.303333333333334
(90,)
C Large 0.18191176470588244
(90,)
D Large -1.143333333333333


### 预测

In [151]:
data3 = data.copy().set_index("product_no")
target = data3.reset_index()[["product_no", "seller_no","warehouse_no"]].drop_duplicates(keep='first')
target = target.set_index("seller_no").sort_index()
target

,product_no,warehouse_no
seller_no,,
seller_1,product_8,wh_3
seller_1,product_2262,wh_1
seller_1,product_2350,wh_1
seller_1,product_2073,wh_1
seller_1,product_2084,wh_1
...,...,...
seller_9,product_2119,wh_12
seller_9,product_2119,wh_15
seller_9,product_2124,wh_12


In [163]:
Result = pd.DataFrame(columns=["seller_no","product_no","warehouse_no","date","qty"])
print(Result)

predict_data = data[["product_no","warehouse_no","date","qty"]].set_index(["product_no","warehouse_no"]).sort_index()
predict_data["date"] = pd.to_datetime(predict_data["date"])
predict_data = predict_data[predict_data["date"]>="2023-04-20"] # 10+15 days before

count = 0
for seller_no, seller_data in target.reset_index().groupby("seller_no"):
    
    seller_mixed_type = seller_type.loc[seller_no]["mixed type"]
    
    if seller_mixed_type not in LinearModels.keys():
        print(f"!!! Unknown {seller_mixed_type} from {seller_no}")
        seller_mixed_type = seller_mixed_type.replace("New", "Large", 1)
    
    res_all = []
    unit = LinearModels[seller_mixed_type]
    
    print(f"    - {seller_no}包含{len(seller_data)}次预测")
    for index, tar in seller_data[["product_no","warehouse_no"]].iterrows():
        product = tar[0]
        wh = tar[1]

        single_series = predict_data.loc[(product, wh)].sort_values(by='date',ascending=True)
        
        single_series["mean 5"] = single_series["qty"].rolling(window=5).mean().fillna(0)
        single_series["std 5"] = single_series["qty"].rolling(window=5).std().fillna(0)
        single_series["mean 15"] = single_series["qty"].rolling(window=15).mean().fillna(0)
        single_series["std 15"] = single_series["qty"].rolling(window=15).std().fillna(0)
        single_series["min 15"] = single_series["qty"].rolling(window=15).min().fillna(0)
        single_series["max 15"] = single_series["qty"].rolling(window=15).max().fillna(0)
        single_series["d mean 5"] = single_series["mean 5"].diff().fillna(0)
        single_series["weekend"] = pd.Index(single_series["date"]).dayofweek.isin([5, 6])
        
        input_data = single_series[["qty","mean 5","std 5","mean 15","std 15","min 15","max 15","d mean 5","weekend"]].values
        input_data = input_data[-WINDOW:].reshape((1,WINDOW*FEATURES))
        
        predict, root = unit.long_term_predict(15, input_data, 1) # first day is Tuesday
        
        if predict is None:
            print(seller_no, product, wh) 
            print(input_data.shape)
        
        predict_series = predict[0][WINDOW:]
        day_series = (single_series["date"][-15:]+pd.Timedelta(days=15)).values
        
        #print(predict_series.shape, day_series.shape)
        
        res = {
            "seller_no" : seller_no,
            "product_no" : product,
            "warehouse_no" : wh,
            "date" : day_series,
            "qty" : predict_series
        }
        res_df = pd.DataFrame(res)
        res_all.append(res_df)

    count+=1
    Result = pd.concat([Result]+res_all, axis=0)
    print(f">>> 预测进度：{count}/{len(target.index.unique())}")
print(">>> 预测完毕")

Empty DataFrame
Columns: [seller_no, product_no, warehouse_no, date, qty]
Index: []
!!! Unknown B New from seller_1
    - seller_1包含12次预测
>>> 预测进度：1/28
    - seller_11包含3次预测
>>> 预测进度：2/28
    - seller_12包含3次预测
>>> 预测进度：3/28
    - seller_13包含2次预测
>>> 预测进度：4/28
    - seller_14包含3次预测
>>> 预测进度：5/28
    - seller_15包含16次预测
>>> 预测进度：6/28
    - seller_17包含5次预测
>>> 预测进度：7/28
    - seller_18包含3次预测
>>> 预测进度：8/28
    - seller_19包含28次预测
>>> 预测进度：9/28
    - seller_20包含2次预测
>>> 预测进度：10/28
    - seller_21包含3次预测
>>> 预测进度：11/28
    - seller_23包含13次预测
>>> 预测进度：12/28
    - seller_26包含2次预测
>>> 预测进度：13/28
    - seller_27包含5次预测
>>> 预测进度：14/28
    - seller_28包含16次预测
>>> 预测进度：15/28
    - seller_29包含4次预测
>>> 预测进度：16/28
    - seller_3包含5次预测
>>> 预测进度：17/28
    - seller_30包含7次预测
>>> 预测进度：18/28
    - seller_31包含8次预测
>>> 预测进度：19/28
    - seller_33包含4次预测
>>> 预测进度：20/28
!!! Unknown A New from seller_34
    - seller_34包含9次预测
>>> 预测进度：21/28
    - seller_35包含11次预测
>>> 预测进度：22/28
    - seller_4包含2次预测
>>> 预测进度：23/28
    - 

- seller_34 为新的A类商户
- seller_1  为新的B类商户

In [162]:
Result.to_excel('result/result2.xlsx', index=False)
Result

,seller_no,product_no,warehouse_no,date,qty
0,seller_1,product_8,wh_3,2023-05-16,1.98
1,seller_1,product_8,wh_3,2023-05-17,2.3
2,seller_1,product_8,wh_3,2023-05-18,2.66
3,seller_1,product_8,wh_3,2023-05-19,2.08
4,seller_1,product_8,wh_3,2023-05-20,2.62
...,...,...,...,...,...
10,seller_9,product_2122,wh_15,2023-05-26,3.86
11,seller_9,product_2122,wh_15,2023-05-27,4.24
12,seller_9,product_2122,wh_15,2023-05-28,4.12
13,seller_9,product_2122,wh_15,2023-05-29,4.12
